In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *

In [ ]:
tcurves = load_tcurves(load_filter_tags())
w_central = central_wavelength(tcurves)

In [ ]:
def identify_lines(line_Arr, qso_flx, nb_min):
    N_fil, N_src = line_Arr.shape
    line_list = []

    for src in range(N_src):
        fil = 0
        this_src_lines = []
        while fil < N_fil:
            this_line = []
            while ~line_Arr[fil, src]:
                fil += 1
                if fil == N_fil: break
            if fil == N_fil: break
            while line_Arr[fil, src]:
                this_line.append(fil)
                fil += 1
                if fil == N_fil: break
            if fil == N_fil: break
            this_src_lines.append(
                fil + np.argmax(qso_flx[np.array(this_line) + nb_min, src])\
                - len(this_line) + 1 + nb_min
            )
        line_list.append(this_src_lines)
    return line_list

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

qso_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
qso_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

In [ ]:
N_sources = qso_flx.shape[1]
N_nb = 6
ew0min = 10

In [ ]:
nb_c_min = 6
nb_c_max = 50
line_qso = np.zeros((nb_c_max - nb_c_min, N_sources)).astype(bool)
cont_est_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
cont_err_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c, tcurves=tcurves)
    cont_est_qso, cont_err_qso = stack_estimation(qso_flx, qso_err, nb_c, N_nb, False)
    line_qso[i] = (
        (qso_flx[nb_c] - cont_est_qso > 3 * (cont_err_qso**2 + qso_err[nb_c]**2) ** 0.5)
        & (qso_flx[nb_c] - cont_est_qso > ew0min * (1 + z_nb) * cont_est_qso / fwhm)
    )
    cont_est_Arr[i] = cont_est_qso
    cont_err_Arr[i] = cont_err_qso
    i += 1

In [ ]:
line_list = identify_lines(line_qso, qso_flx, nb_c_min)

In [ ]:
w_lya = 1215.67
w_SiIV = 1397.61
w_CIV = 1549.48
w_CIII = 1908.73
w_MgII = 2799.12

In [ ]:
no_line = 0
single_line = 0
multiple_lines = 0
right_z = 0
nice_lya = 0
nice_lya_z = 0
for src in range(N_sources):
    try:
        w_obs_lya = w_central[line_list[src][0]]
    except:
        no_line += 1
        continue # No lines here

    if zspec[src] > 2.55:
        right_z += 1

    if len(line_list[src]) == 1:
        single_line += 1
        continue # Just one line here

    multiple_lines += 1

    z_src = w_obs_lya / w_lya - 1

    w_obs_SiIV = (1 + z_src) * w_SiIV
    w_obs_CIV = (1 + z_src) * w_CIV
    w_obs_CIII = (1 + z_src) * w_CIII
    w_obs_MgII = (1 + z_src) * w_MgII

    is_nice_lya = True
    if line_list[src][0] > 27: # Mock zspec max is 4.3
        is_nice_lya = False

    for l in line_list[src][1:]:
        w_obs_l = w_central[l]
        if ~(
            (np.abs(w_obs_l - w_obs_SiIV) < 72.)
            | (np.abs(w_obs_l - w_obs_CIV) < 72.)
            | (np.abs(w_obs_l - w_obs_CIII) < 72.)
            | (np.abs(w_obs_l - w_obs_MgII) < 72.)
            | (w_obs_l > w_obs_MgII)
        ):
            is_nice_lya = False
    if is_nice_lya: nice_lya += 1
    if is_nice_lya & (zspec[src] > 2.5): nice_lya_z += 1

print('N_sources     = {}'.format(N_sources))
print('total_detec   = {}'.format(N_sources - no_line))
print('right_z       = {}'.format(right_z))
print('single_line   = {}'.format(single_line))
print('multiple_line = {}'.format(multiple_lines))
print('nice_lya      = {}'.format(nice_lya))
print('nice_lya_z    = {}'.format(nice_lya_z))

In [ ]:
len(np.where(zspec > 2.5)[0])

In [ ]:
line_list_lens = [len(element) for element in line_list]
plt.hist(line_list_lens)